### 📝 Lab #2 응용 과제: 앙상블 모델 비교 분석

지금까지 배운 랜덤 포레스트, XGBoost, LightGBM, CatBoost 모델을 모두 사용하여 `Heart Failure` 데이터셋에 대한 예측을 수행하고, 어떤 모델이 가장 좋은 성능을 보이는지 비교 분석하는 종합 과제를 수행합니다.

#### 과제 목표

1.  4가지 모델(`RandomForestClassifier`, `XGBClassifier`, `LGBMClassifier`, `CatBoostClassifier`)을 각각 학습시킵니다.
2.  각 모델의 **정확도(Accuracy)**, **F1-Score**, **ROC-AUC** 점수를 계산하여 성능을 비교하는 DataFrame을 생성합니다.
3.  가장 성능이 좋은 모델의 **혼동 행렬(Confusion Matrix)** 을 `Plotly`를 사용하여 시각화하고, 결과를 분석합니다.
4.  가장 성능이 좋은 모델의 **특성 중요도**를 시각화하고, 상위 5개 특성이 무엇인지 설명합니다.

코드작성

In [12]:
# 필요한 라이브러리 임포트
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, confusion_matrix
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import pandas as pd
pd.options.plotting.backend = "plotly"

# 모델 정의
models = {
    'RandomForest': RandomForestClassifier(random_state=42),
    'XGBoost': XGBClassifier(random_state=42),
    'LightGBM': LGBMClassifier(random_state=42),
    'CatBoost': CatBoostClassifier(random_state=42, verbose=False)
}

# 결과를 저장할 딕셔너리
results = {}

# 각 모델 학습 및 평가
for name, model in models.items():
    # 모델 학습
    model.fit(X_train, y_train)
    
    # 예측
    y_pred = model.predict(X_test)
    y_pred_proba = model.predict_proba(X_test)[:, 1]
    
    # 성능 지표 계산
    results[name] = {
        'Accuracy': accuracy_score(y_test, y_pred),
        'F1-Score': f1_score(y_test, y_pred),
        'ROC-AUC': roc_auc_score(y_test, y_pred_proba)
    }

[LightGBM] [Info] Number of positive: 77, number of negative: 162
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000510 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 297
[LightGBM] [Info] Number of data points in the train set: 239, number of used features: 12
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.322176 -> initscore=-0.743791
[LightGBM] [Info] Start training from score -0.743791
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


In [17]:
# 결과를 DataFrame으로 변환
results_df = pd.DataFrame(results).T
print("모델 성능 비교:")
results_df

모델 성능 비교:


,Accuracy,F1-Score,ROC-AUC
RandomForest,0.833333,0.705882,0.891528
XGBoost,0.816667,0.645161,0.831836
LightGBM,0.833333,0.687500,0.848524
CatBoost,0.800000,0.600000,0.888318


In [18]:
results_df['ROC-AUC'].plot.bar()

In [19]:
# 가장 좋은 성능의 모델 찾기
best_model_name = results_df['ROC-AUC'].idxmax()
best_model = models[best_model_name]
best_model

RandomForestClassifier(random_state=42)

In [20]:
# 최고 성능 모델의 혼동 행렬
y_pred_best = best_model.predict(X_test)
cm = confusion_matrix(y_test, y_pred_best)

# 혼동 행렬 시각화
fig = go.Figure(data=go.Heatmap(
    z=cm,
    x=['예측: 0', '예측: 1'],
    y=['실제: 0', '실제: 1'],
    text=cm,
    texttemplate='%{text}',
    textfont={"size": 16},
    colorscale='Blues'
))
fig.update_layout(
    title=f'{best_model_name} 모델의 혼동 행렬',
    xaxis_title='예측 레이블',
    yaxis_title='실제 레이블'
)
fig.show()

In [21]:
# 특성 중요도 시각화
if hasattr(best_model, 'feature_importances_'):
    feature_importance = pd.DataFrame({
        'feature': X.columns,
        'importance': best_model.feature_importances_
    })
else:
    # CatBoost의 경우
    feature_importance = pd.DataFrame({
        'feature': X.columns,
        'importance': best_model.get_feature_importance()
    })


In [22]:
feature_importance = feature_importance.sort_values('importance', ascending=False)

# 상위 5개 특성 출력
print("상위 5개 중요 특성:")
print(feature_importance.head())


상위 5개 중요 특성:
              feature  importance
11               time    0.361383
7    serum_creatinine    0.154118
4   ejection_fraction    0.129121
6           platelets    0.076820
0                 age    0.076779


In [23]:
# 특성 중요도 시각화
fig = px.bar(feature_importance.head(10), 
             x='importance', 
             y='feature',
             title=f'{best_model_name} 모델의 특성 중요도 (상위 10개)',
             orientation='h')
fig.show()